In [ ]:
# pip3 install pycocotools opencv-python #need to install that first

SyntaxError: invalid syntax (3615517132.py, line 1)

In [1]:
import os
import requests
from PIL import Image
from io import BytesIO
from pycocotools.coco import COCO

In [2]:
# Paths
ann_file = '/Users/junwu/Documents/ECS270/Project/Image-Model-with-Grounding-DINO/instances_train2017.json'  # Local annotation JSON
output_dir = './cropped_objects'
os.makedirs(output_dir, exist_ok=True)

# Load COCO
coco = COCO(ann_file)
cat_id_to_name = {cat['id']: cat['name'] for cat in coco.loadCats(coco.getCatIds())}

loading annotations into memory...
Done (t=4.33s)
creating index...
index created!


In [3]:
# Track how many saved per category
saved_count = {cat_id: 0 for cat_id in cat_id_to_name}

max_per_category = 10

# Iterate annotations
for ann in coco.loadAnns(coco.getAnnIds()):
    cat_id = ann['category_id']
    if saved_count[cat_id] >= max_per_category:
        continue  # Already have enough

    x, y, w, h = map(int, ann['bbox'])
    if w < 64 or h < 64:
        continue  # Skip small objects

    img_info = coco.loadImgs(ann['image_id'])[0]
    url = img_info['coco_url']

    try:
        response = requests.get(url, timeout=10)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    except:
        continue  # Skip broken or slow links

    cropped = image.crop((x, y, x + w, y + h))

    label = cat_id_to_name[cat_id]
    filename = f"{label}_{saved_count[cat_id] + 1}.jpg"
    cropped.save(os.path.join(output_dir, filename))
    saved_count[cat_id] += 1

    if all(count >= max_per_category for count in saved_count.values()):
        break  # Done for all categories